# Initialization

Change to `valuenet` directory and add `src` path to `PYTHONPATH`.

In [1]:
%cd /home/ec2-user/SageMaker/valuenet

/home/ec2-user/SageMaker/valuenet


In [2]:
import sys
sys.path.insert(0, '/home/ec2-user/SageMaker/valuenet/src')
sys.path

['/home/ec2-user/SageMaker/valuenet/src',
 '/home/ec2-user/SageMaker/notebook',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python38.zip',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/lib-dynload',
 '',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/site-packages',
 '/home/ec2-user/SageMaker/custom-miniconda/miniconda/envs/valuenet/lib/python3.8/site-packages/IPython/extensions',
 '/home/ec2-user/.ipython']

Read environment

In [3]:
NER_API_SECRET=%env NER_API_SECRET
API_KEY=%env API_KEY
DB_USER=%env DB_USER
DB_PW=%env DB_PW
DB_HOST=%env DB_HOST
DB_PORT=%env DB_PORT
DB_SCHEMA="public"

# Prepare & Preprocess Data

## Add your custom data

TODO..

In [4]:
f = open('data/hack_zurich/handmade_training_data/handmade_data_train.json')

## Transform into Spider representation

In [5]:
%run src/tools/training_data_builder/training_data_builder.py --data hack_zurich

Load data from data/hack_zurich/original/tables.json. N=1
successfully transformed 308 samples for train split
successfully transformed 77 samples for dev split


You will now find your custom data in the two files [data/hack_zurich/original/train.json](data/hack_zurich/original/train.json) and [data/hack_zurich/original/dev.json](data/hack_zurich/original/dev.json).

## Extract Value Candidates using Named Entity Recognition

In [6]:
%run src/named_entity_recognition/api_ner/extract_values.py --data_path=data/hack_zurich/original/train.json --output_path=data/hack_zurich/ner_train.json --ner_api_secret={NER_API_SECRET}

HTTP: 200. for request 'How high is the share of people living in proximity of a busstop in Flaach in the year 2014?'
HTTP: 200. for request 'Which municipality has the highest share of people living in proximity of a busstop?'
HTTP: 200. for request 'Which municipality has the minimum share of people living in proximity of a busstop?'
HTTP: 200. for request 'What are the highest, lowest and average share of people living in proximity of a busstop?'
HTTP: 200. for request 'How many municipalities have a share of people living in proximity of a busstop higher than 89.9 per year?'
HTTP: 200. for request 'How high is the total share of people living in proximity of a busstop in the Canton Zurich in the year 2014?'
HTTP: 200. for request 'Which region had the lowest share of people living in proximity of a busstop in the year 2014?'
HTTP: 200. for request 'How high is the share of people living in proximity of a busstop in Wasterkingen in the year 2005?'
HTTP: 200. for request 'Which munic

In [7]:
%run src/named_entity_recognition/api_ner/extract_values.py --data_path=data/hack_zurich/original/dev.json --output_path=data/hack_zurich/ner_dev.json --ner_api_secret={NER_API_SECRET}

HTTP: 200. for request 'How high is the share of electric vehicles in Niederweningen in the year 2003?'
HTTP: 200. for request 'Which municipality has the highest share of electric vehicles?'
HTTP: 200. for request 'Which municipality has the minimum share of electric vehicles?'
HTTP: 200. for request 'What are the highest, lowest and average share of electric vehicles?'
HTTP: 200. for request 'How many municipalities have a share of electric vehicles higher than 0 per year?'
HTTP: 200. for request 'How high is the total share of electric vehicles in the Canton Zurich in the year 2003?'
HTTP: 200. for request 'Which region had the lowest share of electric vehicles in the year 2003?'
HTTP: 200. for request 'How high is the share of hybrid vehicles within the newly registered cars in Bubikon in the year 2016?'
HTTP: 200. for request 'Which municipality has the highest share of hybrid vehicles within the newly registered cars?'
HTTP: 200. for request 'Which municipality has the minimum sh

## Extract the ground truth values from the SQL query

In [8]:
%run src/tools/get_values_from_sql.py --data_path data/hack_zurich/original/train.json --table_path data/hack_zurich/original/tables.json --ner_path data/hack_zurich/ner_train.json
%run src/tools/get_values_from_sql.py --data_path data/hack_zurich/original/dev.json --table_path data/hack_zurich/original/tables.json --ner_path data/hack_zurich/ner_dev.json

Found values ['Flaach', '2014'] for question: "How high is the share of people living in proximity of a busstop in Flaach in the year 2014?"
Found values [] for question: "Which municipality has the highest share of people living in proximity of a busstop?"
Found values [] for question: "Which municipality has the minimum share of people living in proximity of a busstop?"
Found values [] for question: "What are the highest, lowest and average share of people living in proximity of a busstop?"
Found values ['89.9', '1'] for question: "How many municipalities have a share of people living in proximity of a busstop higher than 89.9 per year?"
Found values ['8', '2014'] for question: "How high is the total share of people living in proximity of a busstop in the Canton Zurich in the year 2014?"
Found values ['4', '2014'] for question: "Which region had the lowest share of people living in proximity of a busstop in the year 2014?"
Found values ['Wasterkingen', '2005'] for question: "How high

This last script doesn't create a new file, but adds the ground truth values to the *ner_dev.json* and *ner_train.json* files, see the new attribute *values*:

```json
    "values": [
      "Wetzikon",
      "2016"
    ]
```

## Pre-processing

In [9]:
%run src/preprocessing/pre_process.py --data_path=data/hack_zurich/original/train.json --ner_data_path=data/hack_zurich/ner_train.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_train.json --database_host={DB_HOST} --database_port={DB_PORT} --database_user={DB_USER} --database_password={DB_PW} --database_schema={DB_SCHEMA}


Process example idx: 0
Question: How high is the share of people living in proximity of a busstop in Flaach in the year 2014?
SQL: SELECT T1.access_by_bus         FROM accessibility_bus AS T1         JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id         WHERE T2.name LIKE 'Flaach' AND T1.year = 2014
Look for potential candidates "[('Flaach', 0.7), ('share', 0.7), ('proximity', 0.7), ('people', 0.7), ('busstop', 0.7), ('2014', 1.0)]" in database hack_zurich (include primary keys: False)
Confirmed the following candidates "[('Flaach', 'name combined', 'city'), ('Flaach', 'name', 'city')]"
Elapsed time is 0.195696 seconds.

Process example idx: 6
Question: Which region had the lowest share of people living in proximity of a busstop in the year 2014?
SQL: SELECT T2.name         FROM accessibility_bus AS T1         JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id         WHERE T2.type_id = 4 AND T1.year = 2014         ORDER BY T1.access_by_bus         LIMIT

In [10]:
%run src/preprocessing/pre_process.py --data_path=data/hack_zurich/original/dev.json --ner_data_path=data/hack_zurich/ner_dev.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_dev.json --database_host={DB_HOST} --database_port={DB_PORT} --database_user={DB_USER} --database_password={DB_PW} --database_schema={DB_SCHEMA}

SQL: SELECT T1.share_of_hybrid_cars         FROM share_hybrid_cars AS T1         JOIN spatialunit AS T2 ON T1.spatialunit_id = T2.spatialunit_id         WHERE T2.name LIKE 'Bauma (bis 2014)' AND T1.year = 2006
Look for potential candidates "[('Bauma', 0.7), ('share', 0.7), ('vehicles', 0.7), ('2014', 1.0), ('2006', 1.0)]" in database hack_zurich (include primary keys: False)
Confirmed the following candidates "[('Bauma', 'name combined', 'city'), ('Bauma', 'name', 'city')]"
Elapsed time is 0.486026 seconds.
Could not find 'Bauma (bis 2014)' in extracted values '['2006', '2014', 'Bauma']'. We add it from the ground truth.
NerExtractionData(heuristic_values_in_quote=[], heuristic_ordinals=[], heuristics_emails=[], heuristics_genders=[], heuristics_null_empty=[], heuristics_variety_common_mentionings=[], heuristics_special_codes=[], heuristics_single_letters=[], heuristics_capitalized_words=['Bauma'], heuristics_months=[], heuristics_location_abbreviations=[], ner_dates=['2014', '2014', '

## Modelling JOINs and SQL-to-SemQL

We start by modeling some JOINs as filters (minor importance, has most probably no effect on your data - you might skip it)

In [11]:
%run src/preprocessing/model_joins_as_filter.py --data_path=data/hack_zurich/preprocessed_train.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_with_joins_train.json 

Load data from data/hack_zurich/original/tables.json. N=1


In [12]:
%run src/preprocessing/model_joins_as_filter.py --data_path=data/hack_zurich/preprocessed_dev.json --table_path=data/hack_zurich/original/tables.json --output=data/hack_zurich/preprocessed_with_joins_dev.json 

Load data from data/hack_zurich/original/tables.json. N=1


And then transform SQL to SemQL

In [13]:
%run src/preprocessing/sql2SemQL.py --data_path data/hack_zurich/preprocessed_with_joins_train.json --table_path data/hack_zurich/original/tables.json --output data/hack_zurich/train.json

Root1(3) Root(3) Sel(0) N(0) A(0) C(20) T(3) Filter(0) Filter(9) A(0) C(12) T(1) V(1) Filter(2) A(0) C(18) T(3) V(0)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(0) A(0) C(20) T(3)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(20) T(3)
Root1(3) Root(5) Sel(0) N(2) A(1) C(20) T(3) A(2) C(20) T(3) A(5) C(20) T(3)
Root1(3) Root(3) Sel(0) N(1) A(0) C(18) T(3) A(3) C(0) T(3) Filter(0) Filter(5) A(0) C(20) T(3) V(0) Filter(2) A(0) C(4) T(1) V(1)
Root1(3) Root(3) Sel(0) N(0) A(0) C(20) T(3) Filter(0) Filter(2) A(0) C(4) T(1) V(2) Filter(2) A(0) C(18) T(3) V(0)
Root1(3) Root(0) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(20) T(3) Filter(0) Filter(2) A(0) C(4) T(1) V(1) Filter(2) A(0) C(18) T(3) V(0)
Root1(3) Root(3) Sel(0) N(0) A(0) C(20) T(3) Filter(0) Filter(9) A(0) C(12) T(1) V(1) Filter(2) A(0) C(18) T(3) V(0)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(0) A(0) C(20) T(3)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(20) T(3)
Root1(3) Root(5) Sel(0) N(2) A(

In [14]:
%run src/preprocessing/sql2SemQL.py --data_path data/hack_zurich/preprocessed_with_joins_dev.json --table_path data/hack_zurich/original/tables.json --output data/hack_zurich/dev.json 

Root1(3) Root(3) Sel(0) N(0) A(0) C(28) T(11) Filter(0) Filter(9) A(0) C(12) T(1) V(0) Filter(2) A(0) C(18) T(11) V(1)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(0) A(0) C(28) T(11)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(28) T(11)
Root1(3) Root(5) Sel(0) N(2) A(1) C(28) T(11) A(2) C(28) T(11) A(5) C(28) T(11)
Root1(3) Root(3) Sel(0) N(1) A(0) C(18) T(11) A(3) C(0) T(11) Filter(0) Filter(5) A(0) C(28) T(11) V(0) Filter(2) A(0) C(4) T(1) V(1)
Root1(3) Root(3) Sel(0) N(0) A(0) C(28) T(11) Filter(0) Filter(2) A(0) C(4) T(1) V(2) Filter(2) A(0) C(18) T(11) V(1)
Root1(3) Root(0) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(28) T(11) Filter(0) Filter(2) A(0) C(4) T(1) V(1) Filter(2) A(0) C(18) T(11) V(0)
Root1(3) Root(3) Sel(0) N(0) A(0) C(29) T(12) Filter(0) Filter(9) A(0) C(12) T(1) V(0) Filter(2) A(0) C(18) T(12) V(1)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(0) A(0) C(29) T(12)
Root1(3) Root(2) Sel(0) N(0) A(0) C(12) T(1) Sup(1) A(0) C(29) T(12)
Root1(3) Root